In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from collections import defaultdict
import math
from sklearn.preprocessing import StandardScaler
from itertools import product
from sklearn import metrics

In [ ]:
from random_dataset import create_random_dataset
from evolutionary_algorithm import EA
from greedy_algorithm import GreedyAlgorithm
from neg_sel import NegativeSelection, load_data

# Peptides

In [ ]:
sampled_peptides_data_dir = "./data/sampled/"
peptides_data_dir = "./data/peptides/"

self1_data = pd.read_csv(peptides_data_dir + "self1-6mers.txt", header=None)

self1_data

## Random Sampling

In [ ]:
random_dataset = create_random_dataset(df=self1_data, n=2000, seed=42)

random_dataset_list = random_dataset[0].to_list()
random_dataset_list

In [ ]:
with open(sampled_peptides_data_dir + "self1_6mers_random.txt", "w") as f:
    for item in random_dataset_list:
        f.write(f"{item}\n")

## Peptide Preperations and Fitness Function

In [ ]:
peptides = self1_data[0].tolist()
peptide_length = len(peptides[0])

position_counts = [defaultdict(int) for _ in range(peptide_length)]

# count frequencies per position
for peptide in peptides:
    for i, aa in enumerate(peptide):
        position_counts[i][aa] += 1

# convert counts to frequencies
position_freqs = []
for pos in position_counts:
    total = sum(pos.values())
    pos_freq = {aa: count / total for aa, count in pos.items()}
    position_freqs.append(pos_freq)


def compute_Fpep(peptide, position_freqs):
    return sum(position_freqs[i].get(aa, 0) for i, aa in enumerate(peptide))


fpep_scores = [compute_Fpep(p, position_freqs) for p in peptides]

fpep_scores

## Greedy Algorithm

In [ ]:
amino_acids = "ACDEFGHIKLMNPQRSTVWY"

motifs = ["".join(motif) for motif in product(amino_acids, repeat=6)]

print(f"Total number of motifs: {len(motifs)}")

In [8]:
len(self1_data[0].tolist())

262000

In [10]:
%%time
# greedy_optimizer = GreedyAlgorithm(
#     peptides=self1_data[0].tolist(),
#     motifs=motifs,
#     t=3,
#     seed=42
# )

# for now, let's limit the lists due to computational constraints
greedy_optimizer = GreedyAlgorithm(peptides=self1_data[0].tolist()[0:200], motifs=motifs[0:200000], t=3, seed=42)
self1_optimized_data = greedy_optimizer.run()
self1_optimized_data

test11
CPU times: total: 35 s
Wall time: 5min 5s


['AAFFAL',
 'AECLQT',
 'AEVCAV',
 'ADEFGV',
 'ACGSRL',
 'AAIALS',
 'AAASSG',
 'AALHRS',
 'AAYTAP',
 'AETFRS',
 'ADSPLK',
 'ADNTND',
 'AEDLMQ',
 'ACPQTV',
 'AEMRPP',
 'AEHLSP',
 'AEQLTY',
 'ADRTAE',
 'AEAVFY',
 'AAELDQ',
 'AEFFSD',
 'ADVTQQ',
 'AAGLAG',
 'ACLCTV',
 'ADIVIN',
 'AESGYG',
 'ACMDTW',
 'AACGAC',
 'ACTDTS',
 'AENFRF',
 'AEYTNS',
 'ADMLCG',
 'ADAVAT',
 'ADFIVK',
 'AEIVKQ',
 'AEGPGP',
 'AELMSS',
 'AAVTPI',
 'AASFSI',
 'IAKNAG',
 'WEKWLS',
 'FAKAYL',
 'LEKHVQ',
 'LEKRDS',
 'EEKIDE',
 'NEKMDL',
 'LEWGLS',
 'MDKPSL',
 'MDKDAE',
 'FDKCPE',
 'VDWSTG',
 'FAWHLD',
 'IDWIND',
 'MEWRNM',
 'VAWMVI',
 'YEWVNP',
 'WEWCIG',
 'YDKTVA',
 'MDKVGK',
 'QEWLVS',
 'YEKLHQ',
 'TEKFSS',
 'IDDEYK',
 'PERYEP',
 'QAQQAL',
 'MAQELG',
 'EEPYQN',
 'GADNID',
 'GEPAEE',
 'SADQQL',
 'GCRSVE',
 'TADDLL',
 'VADGPA',
 'NARPWW',
 'TAQNLS',
 'VEPEEV',
 'KERILQ',
 'PERMTT',
 'IAPDWV',
 'SAPSAP',
 'HARCYE',
 'KDDWLL',
 'VDQIQE',
 'LAPMQG',
 'RAQTTT',
 'YERVSR',
 'PADFTK',
 'PDHSPL',
 'VAPCLV',
 'LAPTGA',
 'PADVVL',

In [10]:
print(f"Size of greedily optimized data set: {len(self1_optimized_data)}")

NameError: name 'self1_optimized_data' is not defined

In [ ]:
with open(sampled_peptides_data_dir + "self1_6mers_greedy.txt", "w") as f:
    for item in self1_optimized_data:
        f.write(f"{item}\n")

## Evolutionary Algorithm

In [ ]:
def fitness(fpep_scores_set):
    if len(fpep_scores_set) == 0:
        return np.inf

    # Minimize avg fpep
    avg_fpep = np.mean(fpep_scores_set)

    # Maximize different kinds of fpep
    different_fpep = np.var(fpep_scores_set)

    # 1 & 0    is the same as what we had before
    weight_avg = 1
    weight_different = -3.0

    return (weight_avg * avg_fpep) + (weight_different * different_fpep)

In [ ]:
## Standard Values across both Algorithms
sigma_val = 30

In [ ]:
ea = EA(data=fpep_scores, N=100, sigma=sigma_val, K=2, p=0.1, mu=0.01, T=1000, f=np.mean, seed=42)
best_individual, best_fitness, fitness_avg, fitness_best = ea.run()

m_ea = EA(data=fpep_scores, N=100, sigma=sigma_val, K=2, p=0.1, mu=0.01, T=1000, f=fitness, seed=42)
multi_best_individual, multi_best_fitness, multi_fitness_avg, multi_fitness_best = m_ea.run()

In [ ]:
plt.plot(fitness_best, label="Best Fitness (f_pep mean)", linestyle="-", color="orange")
plt.plot(fitness_avg, label="Avg Fitness (f_pep mean)", linestyle="--", color="orange")
plt.plot(
    multi_fitness_best, label="Best Fitness (Multi Fitness function, f_pep mean and var)", linestyle="-", color="blue"
)
plt.plot(
    multi_fitness_avg, label="Avg Fitness (Multi Fitness function, f_pep mean and var)", linestyle="--", color="blue"
)
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.title("EA Progress")
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
optimal_peptides = [peptides[i] for i in best_individual]
optimal_peptides

In [ ]:
multi_optimal_peptides = [peptides[i] for i in multi_best_individual]
multi_optimal_peptides

In [ ]:
with open(sampled_peptides_data_dir + "self1_6mers_ea_mean.txt", "w") as f:
    for item in optimal_peptides:
        f.write(f"{item}\n")

with open(sampled_peptides_data_dir + "self1_6mers_ea_multi.txt", "w") as f:
    for item in multi_optimal_peptides:
        f.write(f"{item}\n")

# Improved Evolutionary Algorithm

In [ ]:
peptides = self1_data[0].tolist()
peptide_length = len(peptides[0])

position_counts = [defaultdict(int) for _ in range(peptide_length)]

# count frequencies per position
for peptide in peptides:
    for i, aa in enumerate(peptide):
        position_counts[i][aa] += 1

# convert counts to frequencies
position_freqs = []
for pos in position_counts:
    total = sum(pos.values())
    pos_freq = {aa: count / total for aa, count in pos.items()}
    position_freqs.append(pos_freq)


def compute_Fpep(peptide):
    return sum(position_freqs[i].get(aa, 0) for i, aa in enumerate(peptide))


def compute_aa_frequency_scores(peptides):
    aa_counts = defaultdict(int)
    total_aa = 0

    for p in peptides:
        for aa in p:
            aa_counts[aa] += 1
            total_aa += 1

    aa_freqs = {aa: count / total_aa for aa, count in aa_counts.items()}

    def score(peptide):
        return sum(aa_freqs[aa] for aa in peptide)

    return [score(p) for p in peptides]


aa_freq_scores = compute_aa_frequency_scores(peptides)

feature_vectors = []
for i, peptide in enumerate(peptides):
    fpep = compute_Fpep(peptide)
    aa = aa_freq_scores[i]
    feature_vectors.append([fpep, aa])

scaler = StandardScaler()
feature_vectors = scaler.fit_transform(feature_vectors)


def composite_fitness(subset: np.ndarray):
    fpep_mean = np.mean(subset[:, 0])
    aa_mean = np.mean(subset[:, 1])

    return 1 * fpep_mean + 0.5 * aa_mean


m_ea = EA(data=feature_vectors, N=100, sigma=30, K=2, p=0.1, mu=0.02, T=1000, f=composite_fitness, seed=42)
multi_best_individual, multi_best_fitness, multi_fitness_avg, multi_fitness_best = m_ea.run()

plt.plot(multi_fitness_best, label="Best Fitness (Multi Fitness function, f_pep and aas freq)")
plt.plot(multi_fitness_avg, label="Avg Fitness (Multi Fitness function, f_pep and aas freq)")
plt.xlabel("Generation")
plt.ylabel("Fitness")
plt.title("EA Progress")
plt.legend()
plt.grid(True)
plt.show()

multi_optimal_peptides = [peptides[i] for i in multi_best_individual]
multi_optimal_peptides

In [ ]:
with open(sampled_peptides_data_dir + "self1_6mers_ea_impr.txt", "w") as f:
    for item in multi_optimal_peptides:
        f.write(f"{item}\n")

## Negative Selection

In [ ]:
r_start = 1
r_stop = 5

In [ ]:
ns = NegativeSelection(
    "/data/sampled/6mers.alpha", "./data/sampled/self1_6mers_random.txt", r_start=r_start, r_stop=r_stop
)

ns.run("./data/peptides/ebola-6mers.txt")
ns.run("./data/peptides/self2-6mers.txt")

### Calculate metrics of negative selection

In [ ]:
def preprocess_data(data: pd.DataFrame, r: int):
    anomalous_data = data[data["anomalous"] == 1]
    sensitivity = len(anomalous_data[anomalous_data["score"] > r]) / len(anomalous_data)

    non_anomalous_data = data[data["anomalous"] == 0]
    specificity = len(non_anomalous_data[non_anomalous_data["score"] < r]) / len(non_anomalous_data)

    data["y"] = data["score"] > r

    return anomalous_data, non_anomalous_data, data


def caclulate_roc_auc(data):
    fpr, tpr, thresholds = metrics.roc_curve(data["anomalous"], data["score"])
    auc = metrics.roc_auc_score(data["anomalous"], data["score"])
    roc = metrics.RocCurveDisplay(fpr=fpr, tpr=tpr, roc_auc=auc)

    return auc, roc

In [ ]:
res_dir = "./data/results"
pd.set_option('future.no_silent_downcasting', True) # opt-in for future behavior of replace

fig, ax = plt.subplots(1, r_stop, figsize=(10, 10), constrained_layout=True)

acc_list = []
err_rate_list = []
f1_list = []
precision_list = []
recall_list = []

for r in range(r_start, r_stop + 1):
    anomalous_data = load_data("./data/peptides/ebola-6mers.txt", r, 1)
    normal_data = load_data("./data/peptides/self2-6mers.txt", r, 0)
    data = pd.concat([anomalous_data, normal_data])

    _, _, data = preprocess_data(data, r)
    # display(data.head())

    auc, roc = caclulate_roc_auc(data)
    axis = ax[r - 1]
    roc.plot(ax=axis)
    axis.set_title(f"r={r}")


    y_true = data['anomalous'].to_list()
    y_pred = data['y'].replace({True: 1, False: 0}).to_list()
    acc = metrics.accuracy_score(y_true, y_pred)
    err_rate = 1 - acc

    f1 = metrics.f1_score(y_true, y_pred)
    precision = metrics.precision_score(y_true, y_pred)
    recall = metrics.recall_score(y_true, y_pred)

    acc_list.append(acc)
    err_rate_list.append(err_rate)
    f1_list.append(f1)
    precision_list.append(precision)
    recall_list.append(recall)

In [ ]:
r_values = [r for r in range(r_start, r_stop + 1)]
bar_width = 0.35
x = np.arange(len(r_values))

plt.figure(figsize=(10, 6))

plt.bar(x, acc_list, bar_width, label='Accuracy', color='green')
plt.bar(x + bar_width, err_rate_list, bar_width, label='Error Rate', color='red')

# Add trend lines for accuracy and error rates
# Use moving average for the trend line (smooth line)
window = 1  # Number of points to average over
accuracy_trend = np.convolve(acc_list, np.ones(window)/window, mode='valid')
error_trend = np.convolve(err_rate_list, np.ones(window)/window, mode='valid')

plt.plot(x[window-1:], accuracy_trend, 'g--', label='Accuracy Trend')
plt.plot(x[window-1:], error_trend, 'r--', label='Error Rate Trend')

plt.xlabel('Parameter r')
plt.ylabel('Values')
plt.title('Accuracy vs. Error Rate for Different r Values')
plt.xticks(x + bar_width / 2, r_values)  # Center the x-tick labels
plt.legend()
plt.grid(True, alpha=0.2)

plt.show()

In [ ]:
print()

In [ ]:
bar_width = 0.25

# plt.figure(figsize=(10, 6))
fig, ax = plt.subplots(layout='constrained', figsize=(10, 6))

ax.bar(x, f1_list, bar_width, label='F1 Score', color='blue')
ax.bar(x + bar_width, precision_list, bar_width, label='Precision', color='yellow')
ax.bar(x + 2*bar_width, recall_list, bar_width, label='Recall', color='orange')


ax.set_xlabel('Parameter r')
ax.set_ylabel('Values')
ax.set_title('F1 Score, Precision and Recall for Different r Values')
ax.legend()
ax.set_xticks(x + bar_width, r_values)
ax.grid(True, alpha=0.2)

plt.show()